In [1]:
#imports
using JuMP, LightGraphs, MetaGraphs, CPLEX

## Model Definitions

Here we define the assemblies used for planning. We define an assembly (in practice) as a MetaGraph with a (labelled) vertex for each part & a matrix of disassembly directions for each mate in the assembly. You can run each cell below to generate an assembly model.

In [2]:
#9-part lattice
N = 9; T =6; O=4; R=4; D=2; #Hyperparams for optimization
v = 0:O;
g = MetaGraph(N)
for i = 1:N
    set_prop!(g,i,:name,string(i))
end
add_edge!(g,Edge(1,2)) #define topology
add_edge!(g,Edge(2,3))
add_edge!(g,Edge(1,4))
add_edge!(g,Edge(5,4))
add_edge!(g,Edge(5,2))
add_edge!(g,Edge(6,5))
add_edge!(g,Edge(6,3))
add_edge!(g,Edge(7,4))
add_edge!(g,Edge(8,7))
add_edge!(g,Edge(8,5))
add_edge!(g,Edge(9,8))
add_edge!(g,Edge(9,6))

#dirs
dirs = zeros(Int,N,N)
dirs[2,1] = 1;
dirs[3,2] = 1;
dirs[5,4] = 1;
dirs[6,5] = 1;
dirs[8,7] = 1;
dirs[9,8] = 1;
dirs[1,4] = 2;
dirs[2,5] = 2;
dirs[3,6] = 2;
dirs[4,7] = 2;
dirs[5,8] = 2;
dirs[6,9] = 2;
A = Array(adjacency_matrix(g))
dirs = dirs-dirs';
L = [0 1 -1 -2 2; 0 0 -2 2 -2];

In [21]:
#LEGO Hubble
N = 20; R = 4; O = 4; T = 6;
D=3
v = 0:O;
g = MetaGraph(N)
for i = 1:N
    set_prop!(g,i,:name,string(i))
end
edge_list = [Edge(1,2), Edge(1,3), Edge(1,4), Edge(1,5), Edge(1,6), Edge(6,7), Edge(7,8), Edge(9,1), 
            Edge(10,9), Edge(11,10), Edge(12,13), Edge(12,14), Edge(12,15), Edge(13,8),Edge(16,2),
            Edge(17,4), Edge(18,5), Edge(19,16), Edge(20,17)]
for e in edge_list
    add_edge!(g,e)
end
A = Array(adjacency_matrix(g))
dirs = zeros(Int,N,N)
dirs[2,1] = -1
dirs[3,1] = 2
dirs[4,1] = 1
dirs[5,1] = -2
dirs[6,1] = 3
dirs[7,6] = 3
dirs[8,7] = 3
dirs[9,1] = -3
dirs[10,9] = -3
dirs[11,10] = -3
dirs[13,12] = -3
dirs[14,12] = -3
dirs[15,12] = -3
dirs[13,8] = -2
dirs[16,2] = -1
dirs[17,4] = 1
dirs[18,5] = -2
dirs[19,16] = -1
dirs[20,17] = 1
dirs = dirs - dirs'
L = [0 1 -1 -2 2; 0 0 -2 2 -2];

In [ ]:
#ISS
N = 32;
#=T = round(Int, N/2);=# 
T = 5;
O = 4; 
#=R = round(Int, N/2);=# R = 6
D=3
v = 0:O;
g = MetaGraph(N);
for i = 1:N
    set_prop!(g,i,:name,string(i))
end
edge_list = [Edge(1,2), Edge(2,3), Edge(3,4), Edge(4,5), Edge(6,3), Edge(7,1), Edge(8,7), 
            Edge(9,7), Edge(10,1), Edge(11,10), Edge(12,11), Edge(13,11), Edge(14,13), Edge(15,13),
            Edge(16,2), Edge(17,16), Edge(18,17), Edge(19,17), Edge(20,19), Edge(21,19), Edge(22,19), Edge(23,22), Edge(24,22),
            Edge(25,16), Edge(26,25), Edge(27,25), Edge(28,27), Edge(29,27), Edge(30,27), Edge(31,30), Edge(32,30)]
dirs = zeros(Int,N,N)
dirs[2,1] = 1
dirs[3,2] = 1
dirs[4,3] = 2
dirs[5,4] = 3
dirs[6,3] = -2
dirs[7,1] = 2
dirs[8,7] = -1
dirs[9,7] = 1
dirs[10,1] = -1
dirs[11,10] = -1
dirs[12,11] = -3
dirs[13,11] = -1
dirs[14,13] = -3
dirs[15,13] = 3
dirs[16,2] = 3
dirs[17,16] = 2
dirs[18,17] = -1
dirs[19,17] = 2
dirs[20,19] = -1
dirs[21,19] = 1
dirs[22,19] = 2
dirs[23,22] = -1
dirs[24,22] = 1
dirs[25,16] = -2
dirs[26,25] = -1
dirs[27,25] = -2
dirs[28,27] = -1
dirs[29,27] = 1
dirs[30,27] = -2
dirs[31,30] = -1
dirs[32,30] = 1
for e in edge_list
    add_edge!(g,e)
end
A = Array(adjacency_matrix(g))

In [ ]:
### Chain
N = 50
T = round(Int,N/2); 
O=4; R=10; D=1;
v = 0:O;
g = MetaGraph(N);
for i = 1:N
    set_prop!(g,i,:name,string(i))
end
dirs = zeros(Int,N,N)
for i=1:N-1
    add_edge!(g,Edge(i,i+1))
    dirs[i+1,i] = 1
end

In [ ]:
##Random
using Random
Random.seed!(1312);
#build random graph
N = 500; R=20; S=4;
g = MetaGraph(N);
set_prop!(g,1,:name,string(1))
for i = 2:N
    add_edge!(g,Edge(i,rand(1:i-1)))
    set_prop!(g,i,:name,string(i))
end

## Integer Program
Here we define the IP used in our paper. We implement this using JuMP, a modeling language for opimizations in Julia, and solve using CPLEX. 

*_Note:_* we use JuMP version 0.18.5, CPLEX.jl v0.4.4. This code will *not* work for JuMP v.0.19+

In [4]:
#Variable definitions
cprob = Model(solver=CplexSolver())
@variable(cprob, x[i=1:N,j=0:O,k=1:T], Bin) #orbital positions
@variable(cprob, c[i=1:N,j=1:N,k=0:N,l=1:T], Bin) #connected components
@variable(cprob, a[i=1:N,j=1:N,k=1:T],Bin) #direct adjacency
@variable(cprob, r[i=1:R,j=1:N,k=1:T-1], Bin) #robot assignments
#@variable(cprob, rc[i=1:N,j=1:N,l=1:R,k=1:T-1],Bin) #robot connectivity
@variable(cprob, d[i=1:N,j=-D:D,k=1:T-1],Bin) #directionality
@variable(cprob, b[i=1:N,j=1:N,k=1:T-1], Bin) #breakable variables;

In [5]:
#Constraint Definitions
#-------basics-----------#
@constraint(cprob,[i=1:N,k=1:T],sum(x[i,:,k])==1) #each part must have an orbit
@constraint(cprob,[i=1:N,k=1:T],sum(a[i,:,k])<=N*(1-x[i,0,k]))#disconnected parts iff zero orbit
@constraint(cprob,[i=1:R,k=1:T-1],sum(r[i,:,k])==1) #one part per robot
@constraint(cprob,[i=1:N,k=1:T],(1-x[i,0,k])<=sum(a[i,:,k]))
@constraint(cprob,[i=1:N,j=1:N,t=1:T-1],a[i,j,t+1]<=a[i,j,t]) #adjacency non-increasing
@constraint(cprob, a[:,:,1].==A) #initial condition
@constraint(cprob, [i=1:N,j=1:N,t=1:T],a[i,j,t]==a[j,i,t])
@constraint(cprob,[i=1:N],x[i,0,T]==1)
#-------connectivity--------#
@constraint(cprob,[i=1:N,j=1:N,t=1:T; i!=j],c[i,j,0,t]==a[i,j,t])
@constraint(cprob,[i=1:N,t=1:T],c[i,i,0,t]==1)
@constraint(cprob,[i=1:N,j=1:N,k=1:N,t=1:T],c[i,j,k,t]>=c[i,k,k-1,t]+c[k,j,k-1,t]-1)
@constraint(cprob,[i=1:N,j=1:N,k=1:N,t=1:T],c[i,j,k,t]>=c[i,j,k-1,t])
@constraint(cprob,[i=1:N,j=1:N,k=1:N,t=1:T],c[i,j,k,t]<=c[i,j,k-1,t]+c[i,k,k-1,t])
@constraint(cprob,[i=1:N,j=1:N,k=1:N,t=1:T],c[i,j,k,t]<=c[i,j,k-1,t]+c[k,j,k-1,t])
#------connectivity => orbit-------#
@constraint(cprob,[i=1:N,j=1:N,k=1:O,t=1:T],x[i,k,t]-x[j,k,t]<=(1-a[i,j,t])) #connected parts must be in same orbit
@constraint(cprob,[i=1:N,j=1:N,k=1:O,t=1:T],x[i,k,t]+x[j,k,t]<=1+c[i,j,N,t]) #disconnected parts in different orbits
#------movement--------#
#@constraint(cprob,[i=1:N,j=1:N,l=1:R,t=1:T-1],rc[i,j,l,t]>=c[i,j,N,t+1]+r[l,j,t]-1)
#@constraint(cprob,[i=1:N,j=1:N,l=1:R,t=1:T-1],rc[i,j,l,t]<=c[i,j,N,t+1])
#@constraint(cprob,[i=1:N,j=1:N,l=1:R,t=1:T-1],rc[i,j,l,t]<=r[l,j,t])
#@constraint(cprob,[i=1:N,t=1:T-1],x[i,0,t]+sum(rc[i,:,:,t])>=1)
@constraint(cprob,[i=1:N,j=1:O,t=1:T-1],x[i,j,t]-x[i,j,t+1]<=sum(r[:,i,t])) #one robot per moving part
#------directionality-------#
@constraint(cprob,[i=1:N,k=1:T-1],sum(d[i,:,k])==1) #one direction per part
#can remove j from i: j stationary ^ i moves in D[i,j]
@constraint(cprob,[i=1:N,j=1:N,k=1:T-1],b[i,j,k]>=d[j,0,k]+d[i,dirs[i,j],k]-1)
@constraint(cprob,[i=1:N,j=1:N,k=1:T-1],b[i,j,k]>=d[j,dirs[j,i],k]+d[i,dirs[i,j],k]-1)
@constraint(cprob,[i=1:N,j=1:N,k=1:T-1],b[i,j,k]<=d[j,0,k]+d[j,dirs[j,i],k])
@constraint(cprob,[i=1:N,j=1:N,k=1:T-1],b[i,j,k]<=d[i,dirs[i,j],k])

#@constraint(cprob,d[1,-1,1]==1)
#@constraint(cprob,d[2,0,1]==1)

@constraint(cprob,[i=1:N,j=1:N,k=1:T-1,l=-D:D,i!=j],d[i,l,k]-d[j,l,k]<=1-c[i,j,N,k+1]) #connected parts move in same dir
@constraint(cprob,[i=1:N,j=1:O,k=1:T-1],x[i,j,k+1]-x[i,j,k]<=1-d[i,0,k]) #stationary parts remain in orbital, unless going to depot
@constraint(cprob,[i=1:N,j=-D:-1,k=1:T-1],d[i,j,k]<=sum(r[:,i,k]))
@constraint(cprob,[i=1:N,j=1:D,k=1:T-1],d[i,j,k]<=sum(r[:,i,k]))
#@constraint(cprob,[i=1:N,k=1:T-1],m[i,k]<=O*(d[i,0,k]+x[i,0,k+1]))
#@constraint(cprob,[i=1:N,k=1:T-1],m[i,k]>=sum(d[i,-D:-1, k]))#+sum(d[i,1:D,k]))
#@constraint(cprob,[i=1:N,k=1:T-1],m[i,k]>=sum(d[i,1:D, k]))

@constraint(cprob,[i=1:N,j=1:N,k=1:T-1],a[i,j,k]-a[i,j,k+1]<=b[i,j,k]+b[j,i,k]);

In [7]:
#Minimum-Time Planning
@variable(cprob, f[t=1:T],Bin) #denotes if finished
@constraint(cprob,[t=1:T],f[t]<=sum(x[:,1:O,t]))
@constraint(cprob,[i=1:N,j=1:O,t=1:T],f[t]>=x[i,j,t])
@objective(cprob, Min, sum(f)) #minimize final time
status = solve(cprob)

2 of 3 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 5.0000.
Tried aggregator 3 times.
MIQP Presolve eliminated 17231 rows and 3290 columns.
MIQP Presolve modified 1538 coefficients.
Aggregator did 901 substitutions.
Reduced MIQP has 9758 rows, 2241 columns, and 28645 nonzeros.
Reduced MIQP has 2241 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (67.52 ticks)
Probing fixed 24 vars, tightened 0 bounds.
Probing changed sense of 326 constraints.
Probing time = 0.05 sec. (19.22 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 1044 rows and 128 columns.
MIP Presolve modified 69 coefficients.
Aggregator did 261 substitutions.
Reduced MIP has 8453 rows, 1852 columns, and 25018 nonzeros.
Reduced MIP has 1852 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.07 sec. (46.68 ticks)
Probing fixed 18 vars, tightened 0 bounds.
Probing changed sense of 145 constraints.
Probing time = 0.04 sec. (16.38 ticks)
Tri

:Optimal

In [6]:
#Minimum-Fuel Planning
obj = 0;
for i = 1:N
    for t = 1:T-1
        obj += (x[i,:,t+1]-x[i,:,t])'*L'*L*(x[i,:,t+1]-x[i,:,t])
    end
end
@objective(cprob, Min, obj) #minimize distance travelled
status = solve(cprob)

Tried aggregator 3 times.
MIP Presolve eliminated 17025 rows and 3288 columns.
MIP Presolve added 1692 rows and 846 columns.
MIP Presolve modified 1465 coefficients.
Aggregator did 869 substitutions.
Reduced MIP has 11466 rows, 3115 columns, and 31829 nonzeros.
Reduced MIP has 3115 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (64.73 ticks)
Probing fixed 204 vars, tightened 0 bounds.
Probing changed sense of 366 constraints.
Probing time = 0.07 sec. (23.82 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 1998 rows and 404 columns.
MIP Presolve modified 33 coefficients.
Aggregator did 302 substitutions.
Reduced MIP has 9166 rows, 2393 columns, and 26469 nonzeros.
Reduced MIP has 2393 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (50.14 ticks)
Probing fixed 25 vars, tightened 0 bounds.
Probing changed sense of 154 constraints.
Probing time = 0.05 sec. (19.65 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 652 rows a

:Optimal

## Heuristic Solutions
Below we include our implementation of the heuristic strategies outlined in our paper. 

In [18]:
include("heuristic-helpers.jl") #include helper functions

mutable struct Assembly #Type which defines assembly state
    graph::MetaDiGraph
    singles #list of leaves w/ assigned robots
    threads #list of interior nodes w/ assigned robots
end

function heuristic_policy(const_state,R,S) ##High-level wrapper, takes in construction state & returns policy
    #construction state is list of assems
    flag = true
    nr = 0
    na = length(const_state)
    for assem in const_state
        (flag, nr) = initAssign(assem,nr,R)
    end
    while flag #loop while policy changing
        flag = false
        assem_order = sortperm([nv(const_state[k].graph) for k in 1:length(const_state)])
        for assem in const_state[assem_order]#for each assembly
            singles_order = sortperm([props(assem.graph,assem.singles[k])[:layer] for k in 1:length(assem.singles)])
            for s in assem.singles[singles_order]
                (success, nr,na) = growSingle!(assem,s,nr,na,R,S) #try to grow
                if success
                    flag = true #if possible, set flag to true
                end
            end
            thread_order = sortperm([props(assem.graph,assem.threads[k])[:layer] for k in 1:length(assem.threads)])
            for t in assem.threads[thread_order]
                (success, nr,na) = growThread!(assem,t,nr,na,R,S) #try to grow
                if success
                    flag = true #if possible, set flag
                end
            end
        end
    end 
    new_assems = []
    for assem in const_state
        append!(new_assems,partitionAssem(assem))
    end
    return new_assems
end

function initAssign(assem,nr,R) #function which returns initial assignments to all leaves
    for n = sortperm([props(assem.graph,i)[:layer] for i in 1:nv(assem.graph)])
        if props(assem.graph,n)[:weight] == 1 && nr < R
            push!(assem.singles,n)
            nr += 1
        end
    end
    root = findfirst([length(inneighbors(assem.graph,i)) for i in 1:nv(assem.graph)].==0)
    if length(outneighbors(assem.graph,root)) == nv(assem.graph) - 1 && nr < R
        push!(assem.singles,root)
        nr += 1
    end
    return (nr<R, nr)
end

function growSingle!(assem,s,nr,na,R,S) #grows single part into assembly
    if nv(assem.graph) <= 3 || nv(assem.graph) == length(assem.singles)
        return (false,nr,na)
    else
        parent = inneighbors(assem.graph,s)[1]
        if na < S && nr + numAdded(assem,parent) <= R && props(assem.graph,parent)[:weight] <= Int(floor(nv(assem.graph)/(length(assem.singles)+length(assem.threads)+1)))
            na += 1
            nr += numAdded(assem,parent)
            push!(assem.threads,parent)
            for child in outneighbors(assem.graph,parent)
                deleteat!(assem.singles,assem.singles.==child)
                if any(assem.threads.==child)
                    deleteat!(assem.threads,assem.threads.==child)
                    na -= 1
                end
            end
            return (true,nr,na)
        else 
            return (false,nr,na)
        end
    end
end

function growThread!(assem,t,nr,na,R,S) #grows thread further into assembly, merging & bookkeeping as necessary
    parent = inneighbors(assem.graph,t)[1]
    if nr + numAdded(assem,parent) <= R && props(assem.graph,parent)[:weight] <= Int(floor(nv(assem.graph)/(length(assem.singles)+length(assem.threads)+1)))
        nr += numAdded(assem,parent)
        na += 1
        push!(assem.threads,parent)
        for child in outneighbors(assem.graph,parent)
            deleteat!(assem.singles,assem.singles.==child)
            if any(assem.threads.==child)
                deleteat!(assem.threads,assem.threads.==child)
                na -= 1
            end
        end
        return (true,nr,na)
    else 
        return (false,nr,na)
    end
end

function partitionAssem(assem) #function which creates new assem state s' after heuristic termination
    #removes singles & creates new assems from thread
    new_assems = []
    for t in assem.threads
        n = 1
        curr = MetaGraph(1)
        set_props!(curr,n,Dict(:name=>props(assem.graph,t)[:name]))
        queue = [t]
        while length(queue) > 0
            v = popfirst!(queue)
            v_ind = findfirst([props(curr,i)[:name] for i in 1:nv(curr)].==props(assem.graph,v)[:name])
            for child in outneighbors(assem.graph,v)
                n+=1
                add_vertex!(curr)
                set_props!(curr,n,Dict(:name=>props(assem.graph,child)[:name]))
                add_edge!(curr,v_ind,n)
                push!(queue,child)
            end
        end
        push!(new_assems,Assembly(label(curr),[],[]))
    end
    root = findfirst([length(inneighbors(assem.graph,i)) for i in 1:nv(assem.graph)].==0)
    if !(root in assem.singles)    
        n = 1
        curr = MetaGraph(1)
        set_props!(curr,n,Dict(:name=>props(assem.graph,root)[:name]))
        queue = [root]
        while length(queue) > 0
            v = popfirst!(queue)
            v_ind = findfirst([props(curr,i)[:name] for i in 1:nv(curr)].==props(assem.graph,v)[:name])
            for child in outneighbors(assem.graph,v)
                if !(child in assem.threads) && !(child in assem.singles)
                    n+=1
                    add_vertex!(curr)
                    set_props!(curr,n,Dict(:name=>props(assem.graph,child)[:name]))
                    add_edge!(curr,v_ind,n)
                    push!(queue,child)
                end
            end
        end
        push!(new_assems,Assembly(label(curr),[],[]))
    end
    return new_assems
end

partitionAssem (generic function with 1 method)

In [22]:
##Heuristic Runs -- NOTE these won't work for the lattice example
assem = Assembly(label(g),[],[])
s = [assem]
states = [s]
tstop = 100
@time states,T = fast_heuristic_run(deepcopy(s),R,O,tstop) #wrapper (stored in heuristic-helpers.jl)
@show T
@time states,T = greedy_heuristic_run(deepcopy(s),R,O,tstop)
@show T

  0.001790 seconds (7.90 k allocations: 1.120 MiB)
T = 8
  0.017408 seconds (7.28 k allocations: 1.039 MiB, 91.49% gc time)
T = 8


8

In [23]:
#Helper function to assign parts to locations (greedily) & return total distance traveled
using LinearAlgebra
#L = [0 0 -1 -2 2
     #0 1 -2 2 -2]
L = [0 1 -1 -1 1; 0 1 1 -1 -1]
dists = zeros(5,5)
costs = zeros(5,5)
for i = 1:5
    for j = 1:5
        dists[i,j] = norm(L[:,i]-L[:,j])#+norm(L[:,j])
    end
end
for i = 1:5
    for j = 1:5
        costs[i,j] = norm(L[:,i]-L[:,j])+norm(L[:,j])
    end
end

function process_dist(states)
    locs = zeros(Int,N,length(states))
    locs[:,1] = ones(Int,N,1)
    total_cost = 0;
    for t in 2:length(states)
        assem_order= sortperm([nv(assem.graph) for assem in states[t]])
        possible = collect(1:4)
        for assem in states[t][assem_order]
            curr_loc = locs[parse(Int,props(assem.graph,1)[:name]),t-1]
            next_loc = possible[argmin(costs[curr_loc,possible.+1])]
            for k in [parse(Int,props(assem.graph,i)[:name]) for i in 1:nv(assem.graph)]
                locs[k,t] = next_loc
            end
            deleteat!(possible,possible.==next_loc)
        end
        stage_cost = 0;
        for i in 1:N
            stage_cost += dists[locs[i,t-1]+1,locs[i,t]+1]
        end
        total_cost+= stage_cost
    end
    @show locs[:,2]
    return total_cost
end

process_dist(states)

locs[:, 2] = [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1]


28.28427124746191